Task 1

In [4]:
import json
import requests

API_ENDPOINT = "http://www.wikidata.org/w/api.php?"
ACM_AWARD = "Q185667"
AWARD_RECIPIENTS_PROPERTIES = dict (action='wbsearchentities', 
                                    format='json',
                                    language='en',
                                    type='property',
                                    search='award received')


In [5]:
def get_turing_award_recipients():
    response = requests.get(API_ENDPOINT, AWARD_RECIPIENTS_PROPERTIES)

    print(response.json())